In [1]:
# import numpy as np
import pandas as pd

In [2]:
dfile = pd.read_csv("BMS Requirements.csv")
dfile

,Req ID,Req Name,Description,Pre-requisite,Cost,Value,Test Cases,Importance
0,101,URL,Unique web address to access the website,0.0,8.0,9.0,3.0,H
1,102,Choose Region Modal,To choose the city inorder to proceed,101.0,3.0,2.0,3.0,H
2,103,Home Page,Contains various...,102.0,7.0,8.0,5.0,H
3,104,Change Region,Enables to change the city,103.0,4.0,3.0,1.0,M
4,105,Search,To perform search,103.0,6.0,7.0,2.0,H
5,106,Search Results,Displays the search results,105.0,7.0,8.0,1.0,H
6,107,Login Button,Sign in to the system for existing users,103.0,6.0,3.0,4.0,L
7,108,Forgot Password,To enable sign in when user forgets password,107.0,5.0,4.0,0.0,L
8,109,Continue with Facebook,To login with Facebook account,107.0,7.0,1.0,0.0,L
9,110,Continue with Google,To login with Google account,107.0,7.0,3.0,0.0,L


In [3]:
dfile['Pre-requisite'].fillna(0, inplace=True)
dfile.head()

,Req ID,Req Name,Description,Pre-requisite,Cost,Value,Test Cases,Importance
0,101,URL,Unique web address to access the website,0.0,8.0,9.0,3.0,H
1,102,Choose Region Modal,To choose the city inorder to proceed,101.0,3.0,2.0,3.0,H
2,103,Home Page,Contains various...,102.0,7.0,8.0,5.0,H
3,104,Change Region,Enables to change the city,103.0,4.0,3.0,1.0,M
4,105,Search,To perform search,103.0,6.0,7.0,2.0,H


In [4]:
dfile['Pre-requisite'].unique()

array([  0., 101., 102., 103., 105., 107., 112., 114., 127., 133., 136.,
       137., 138., 139., 142., 143.])

In [5]:
PRE_REQ_W = [0]*dfile['Req ID'].count()
dfile['PRE_REQ_WEIGHT'] = PRE_REQ_W
dfile.head()

,Req ID,Req Name,Description,Pre-requisite,Cost,Value,Test Cases,Importance,PRE_REQ_WEIGHT
0,101,URL,Unique web address to access the website,0.0,8.0,9.0,3.0,H,0
1,102,Choose Region Modal,To choose the city inorder to proceed,101.0,3.0,2.0,3.0,H,0
2,103,Home Page,Contains various...,102.0,7.0,8.0,5.0,H,0
3,104,Change Region,Enables to change the city,103.0,4.0,3.0,1.0,M,0
4,105,Search,To perform search,103.0,6.0,7.0,2.0,H,0


In [6]:
for i in dfile['Pre-requisite']:
    if (i in dfile['Req ID'].values):
        print(i," is a Pre-requisite", end=" ")
        x = dfile.loc[dfile['Req ID'] == i].index.values[0]
        print("at row: ",x, end='\n')
        dfile['PRE_REQ_WEIGHT'][x] = dfile['PRE_REQ_WEIGHT'][x]+1

101.0  is a Pre-requisite at row:  0
102.0  is a Pre-requisite at row:  1
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
105.0  is a Pre-requisite at row:  4
103.0  is a Pre-requisite at row:  2
107.0  is a Pre-requisite at row:  6
107.0  is a Pre-requisite at row:  6
107.0  is a Pre-requisite at row:  6
107.0  is a Pre-requisite at row:  6
107.0  is a Pre-requisite at row:  6
103.0  is a Pre-requisite at row:  2
112.0  is a Pre-requisite at row:  11
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
103.0  is a Pre-requisite at row:  2
114.0  is a Pre-requisite at row:  13
127.0  is a Pre-requisite at row:  2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
dfile.head()

,Req ID,Req Name,Description,Pre-requisite,Cost,Value,Test Cases,Importance,PRE_REQ_WEIGHT
0,101,URL,Unique web address to access the website,0.0,8.0,9.0,3.0,H,1
1,102,Choose Region Modal,To choose the city inorder to proceed,101.0,3.0,2.0,3.0,H,1
2,103,Home Page,Contains various...,102.0,7.0,8.0,5.0,H,16
3,104,Change Region,Enables to change the city,103.0,4.0,3.0,1.0,M,0
4,105,Search,To perform search,103.0,6.0,7.0,2.0,H,1


In [8]:
# Priority = [0]*dfile['Req ID'].count()
# dfile['Priority'] = Priority
# dfile